<a href="https://colab.research.google.com/github/changqicode/final-project/blob/main/final_project_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

divide the audio file

In [ ]:
from pydub import AudioSegment

audio = AudioSegment.from_wav("/content/final project/bubble.wav")

time_segments = [
    (56, 76),    # 1/4
    (85, 105),   # 1/2
    (115, 135),   # 3/4
    (145, 165)   # 1
]

output_names = ["1_4", "1_2", "3_4", "1"]

for (start, end), name in zip(time_segments, output_names):
    cut_audio = audio[start*1000:end*1000]
    cut_audio.export(f"{name}.wav", format="wav")

print("完成切割并输出分数形式命名的文件！")


完成切割并输出分数形式命名的文件！


# data analysis


In [ ]:
!pip -q install librosa soundfile


In [ ]:
# 如果还没装依赖，先运行：!pip -q install librosa soundfile

import os, glob, numpy as np, pandas as pd, matplotlib.pyplot as plt
import librosa, scipy.signal as sps

# ===================== 路径与文件 =====================
DATA_DIR = "/content"             # Colab 工作区
PLANTS_SUBDIR = "plants"          # /content/plants/*.wav
OUTPUT_DIR = "/content/outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 四段主音频（自动兜底 1power 或 1）
MAIN_FILES = {"1/4":["/content/bubble sounds/1_4.wav"], "1/2":["/content/bubble sounds/1_2.wav"], "3/4":["/content/bubble sounds/3_4.wav"], "1":["/content/bubble sounds/1.wav"]}
labels_order = ["plants", "1/4", "1/2", "3/4", "1"]  # 画图时把 plants 放最左

# ===================== 分析参数 =====================
TRIM_PLANTS_FIRST_SECONDS = 10.0   # plants 去掉前 10s
MIN_SECONDS_AFTER_TRIM    = 3.0    # 剪后不足 3s 的样本跳过
TARGET_SR = None                   # 保留原采样率（librosa）

# 频段（可选；否则全频）
# 例如：FMIN=20000; FMAX=100000（需采样率足够）
FMIN = None
FMAX = None

# STFT 参数
N_FFT = 2048
HOP_LENGTH = 512

FIGSIZE = (8,5)
np.random.seed(0)  # 让散点抖动可复现

print("DATA_DIR:", os.listdir(DATA_DIR))
print("plants 数量（原始）:", len(glob.glob(os.path.join(DATA_DIR, PLANTS_SUBDIR, "*.wav"))) if os.path.isdir(os.path.join(DATA_DIR, PLANTS_SUBDIR)) else 0)

# ===================== 工具函数（仅 RMS/Hf/ACI） =====================
def load_audio(path, target_sr=TARGET_SR):
    # 不做逐文件幅度归一化
    y, sr = librosa.load(path, sr=target_sr, mono=True)
    return y, sr

def trim_first_seconds(y, sr, seconds):
    start = int(seconds * sr)
    return y[start:] if start < len(y) else np.array([], dtype=y.dtype)

def bandlimit(y, sr, fmin=FMIN, fmax=FMAX):
    if len(y) == 0 or (fmin is None and fmax is None):
        return y
    nyq = sr / 2
    if fmin is not None and fmax is not None:
        lo = max(1e-6, min(0.999,  fmin/nyq))
        hi = max(lo+1e-6, min(0.9999, fmax/nyq))
        sos = sps.butter(6, [lo, hi], btype='bandpass', output='sos')
    elif fmin is not None:
        lo = max(1e-6, min(0.999, fmin/nyq))
        sos = sps.butter(6, lo, btype='highpass', output='sos')
    else:
        hi = max(1e-6, min(0.9999, fmax/nyq))
        sos = sps.butter(6, hi, btype='lowpass', output='sos')
    return sps.sosfiltfilt(sos, y)

def compute_rms(y):
    if len(y) == 0: return np.nan
    return float(np.sqrt(np.mean(y**2)))

def spectral_entropy_Hf(y, sr, n_fft=N_FFT, hop_length=HOP_LENGTH, fmin=FMIN, fmax=FMAX):
    if len(y) == 0: return np.nan
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
    if fmin is not None or fmax is not None:
        mask = np.ones_like(freqs, dtype=bool)
        if fmin is not None: mask &= (freqs >= fmin)
        if fmax is not None: mask &= (freqs <= fmax)
        S = S[mask, :]
        if S.size == 0: return np.nan
    P = np.mean(S, axis=1).astype(float)
    if np.sum(P) <= 0: return np.nan
    P = P / np.sum(P)
    H = -np.sum(P * np.log(P + 1e-12))
    return float(H / np.log(len(P)))

def acoustic_complexity_index(y, sr, n_fft=N_FFT, hop_length=HOP_LENGTH, fmin=FMIN, fmax=FMAX):
    if len(y) == 0: return np.nan
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
    if fmin is not None or fmax is not None:
        mask = np.ones_like(freqs, dtype=bool)
        if fmin is not None: mask &= (freqs >= fmin)
        if fmax is not None: mask &= (freqs <= fmax)
        S = S[mask, :]
        if S.size == 0: return np.nan
    diff = np.abs(np.diff(S, axis=1))
    num = np.sum(diff, axis=1)
    den = np.sum(S[:, :-1], axis=1) + 1e-12
    return float(np.sum(num / den))

def analyze_series(y, sr):
    """只返回：RMS, Hf, ACI"""
    y_f = bandlimit(y, sr, FMIN, FMAX)
    return {
        "RMS": compute_rms(y_f),
        "Hf":  spectral_entropy_Hf(y_f, sr),
        "ACI": acoustic_complexity_index(y_f, sr),
    }

def analyze_file(path):
    y, sr = load_audio(path, target_sr=TARGET_SR)
    return analyze_series(y, sr)

def pick_first_existing(base_dir, candidates):
    for name in candidates:
        p = os.path.join(base_dir, name)
        if os.path.exists(p):
            return p
    return None

# ===================== 主音频（四段） =====================
main_rows = []
for label in ["1/4", "1/2", "3/4", "1"]:
    path = pick_first_existing(DATA_DIR, MAIN_FILES[label])
    print(f"{label} -> {path}")
    met = {"RMS": np.nan, "Hf": np.nan, "ACI": np.nan} if path is None else analyze_file(path)
    main_rows.append({"label": label, **met})
df_main = pd.DataFrame(main_rows).set_index("label")
print("\n四段主音频指标：")
display(df_main)

# ===================== plants（批量；剪前10s；不足3s跳过） =====================
plants_dir = os.path.join(DATA_DIR, PLANTS_SUBDIR)
plants_paths = glob.glob(os.path.join(plants_dir, "*.wav")) if os.path.isdir(plants_dir) else []
print("\nplants 文件数（原始）:", len(plants_paths))

rows, kept = [], 0
for p in plants_paths:
    y, sr = load_audio(p, target_sr=TARGET_SR)
    y = trim_first_seconds(y, sr, TRIM_PLANTS_FIRST_SECONDS)
    if len(y) < int(MIN_SECONDS_AFTER_TRIM * sr):
        continue
    rows.append(analyze_series(y, sr)); kept += 1

plants_df = pd.DataFrame(rows)
print("plants 文件数（保留）:", kept)
if kept > 0:
    display(plants_df.describe())

metric_cols = ["RMS", "Hf", "ACI"]

if kept == 0:
    plants_stats = pd.Series({m: np.nan for m in metric_cols})
    plants_sd    = pd.Series({m: np.nan for m in metric_cols})
else:
    plants_stats = plants_df[metric_cols].mean(numeric_only=True)
    plants_sd    = plants_df[metric_cols].std(ddof=1, numeric_only=True)

plants_se   = plants_sd / np.sqrt(max(kept,1))
plants_ci95 = 1.96 * plants_se

# ===================== 汇总与效应量（Glass's Δ） =====================
df_main_use = df_main[metric_cols].copy()
glass_delta = (df_main_use - plants_stats) / (plants_sd.replace(0, np.nan))
glass_delta = glass_delta.rename(columns={c: f"{c}_GlassΔ" for c in glass_delta.columns})

# 保存 CSV（含 plants 均值/SD/CI）
summary = df_main_use.copy()
for c in metric_cols:
    summary[f"plants_mean_{c}"] = plants_stats[c]
    summary[f"plants_sd_{c}"]   = plants_sd[c]
    summary[f"plants_se_{c}"]   = plants_se[c]
    summary[f"plants_ci95_{c}"] = plants_ci95[c]
summary_full = pd.concat([summary, pd.DataFrame([plants_stats], index=["plants"])], axis=0)
csv_path = os.path.join(OUTPUT_DIR, "summary_RMS_Hf_ACI.csv")
summary_full.to_csv(csv_path, encoding="utf-8-sig")
print("\n已保存 CSV：", csv_path)

# ===================== 作图（叠加 plants 散点云 + 箱线图；GlassΔ；原值差异） =====================
def plot_with_plants_distribution(metric, df_main_use, plants_vals, ylabel, save_name):
    """
    x 轴：plants（0） + 1/4,1/2,3/4,1（1..4）
    plants：散点云（抖动）+ 箱线图
    功率：连线散点
    """
    order = ["1/4", "1/2", "3/4", "1"]
    x_power = np.arange(1, len(order)+1)  # 1..4
    y_power = df_main_use.loc[order, metric].values

    plt.figure(figsize=FIGSIZE)
    # plants 箱线 + 散点
    if plants_vals is not None and len(plants_vals) > 0 and np.isfinite(plants_vals).any():
        plt.boxplot([plants_vals], positions=[0], widths=0.22, showfliers=False)
        jitter = np.random.normal(loc=0, scale=0.03, size=len(plants_vals))
        plt.scatter(jitter, plants_vals, alpha=0.35, s=12)

    # 功率级别：连线散点
    plt.plot(x_power, y_power, marker='o')

    plt.xticks([0,1,2,3,4], ["plants", "1/4", "1/2", "3/4", "1"])
    plt.xlim(-0.5, 4.5)
    plt.ylabel(ylabel)
    plt.title(f"{metric}: power trend with plants distribution")
    plt.tight_layout()
    out_path = os.path.join(OUTPUT_DIR, save_name)
    plt.savefig(out_path, dpi=200); plt.close()
    print("保存：", out_path)

def plot_glass_delta(metric, glass_df, save_name):
    order = ["1/4", "1/2", "3/4", "1"]
    x = np.arange(len(order))
    col = f"{metric}_GlassΔ"
    y = glass_df.loc[order, col].values

    plt.figure(figsize=FIGSIZE)
    plt.plot(x, y, marker='o')
    for ref in [0, 0.2, 0.5, 0.8, -0.2, -0.5, -0.8]:
        ls = '-' if ref == 0 else '--'
        plt.axhline(ref, linestyle=ls, linewidth=1)
    plt.xticks(x, order)
    plt.ylabel("Glass's Δ")
    plt.title(f"{metric}: standardized difference vs plants")
    plt.tight_layout()
    out_path = os.path.join(OUTPUT_DIR, save_name)
    plt.savefig(out_path, dpi=200); plt.close()
    print("保存：", out_path)

def plot_raw_diff(metric, df_main_use, plants_mean, ylabel, save_name):
    order = ["1/4", "1/2", "3/4", "1"]
    x = np.arange(len(order))
    y = df_main_use.loc[order, metric].values - float(plants_mean)

    plt.figure(figsize=FIGSIZE)
    plt.axhline(0, linestyle='-', linewidth=1)
    plt.plot(x, y, marker='o')
    plt.xticks(x, order)
    plt.ylabel(f"{ylabel} minus plants_mean")
    plt.title(f"{metric}: raw difference vs plants mean")
    plt.tight_layout()
    out_path = os.path.join(OUTPUT_DIR, save_name)
    plt.savefig(out_path, dpi=200); plt.close()
    print("保存：", out_path)

# plants 值数组（用于散点云/箱线图）
plants_vals_dict = {}
if kept > 0:
    for m in metric_cols:
        plants_vals_dict[m] = plants_df[m].dropna().values
else:
    for m in metric_cols:
        plants_vals_dict[m] = np.array([])

ylabels = {"RMS":"RMS", "Hf":"Hf", "ACI":"ACI"}

for metric in metric_cols:
    # 趋势 + plants 分布
    plot_with_plants_distribution(
        metric,
        df_main_use,
        plants_vals_dict.get(metric, np.array([])),
        ylabel=ylabels[metric],
        save_name=f"trend_{metric}_with_plants_dist.png"
    )
    # Glass’s Δ
    plot_glass_delta(metric, glass_delta, save_name=f"glass_delta_{metric}.png")
    # 原值差异
    plot_raw_diff(metric, df_main_use, plants_stats[metric], ylabels[metric], save_name=f"raw_diff_{metric}.png")

print("\n所有结果输出目录：", OUTPUT_DIR)


DATA_DIR: ['.config', '.ipynb_checkpoints', 'outputs', 'plants', 'bubble sounds', 'sample_data']
plants 数量（原始）: 269
1/4 -> /content/bubble sounds/1_4.wav
1/2 -> /content/bubble sounds/1_2.wav
3/4 -> /content/bubble sounds/3_4.wav
1 -> /content/bubble sounds/1.wav

四段主音频指标：


,RMS,Hf,ACI
label,,,
1/4,0.000736,0.603451,534.174072
1/2,0.001301,0.646659,664.278381
3/4,0.004543,0.621876,761.269409
1,0.005499,0.593900,761.739990



plants 文件数（原始）: 269
plants 文件数（保留）: 269


,RMS,Hf,ACI
count,269.000000,269.000000,269.000000
mean,0.000332,0.626830,461.243349
std,0.000041,0.012330,0.467430
min,0.000308,0.526144,458.523346
25%,0.000318,0.627209,460.973999
50%,0.000323,0.629706,461.181335
75%,0.000330,0.631627,461.401672
max,0.000709,0.636232,464.163513



已保存 CSV： /content/outputs/summary_RMS_Hf_ACI.csv
保存： /content/outputs/trend_RMS_with_plants_dist.png
保存： /content/outputs/glass_delta_RMS.png
保存： /content/outputs/raw_diff_RMS.png
保存： /content/outputs/trend_Hf_with_plants_dist.png
保存： /content/outputs/glass_delta_Hf.png
保存： /content/outputs/raw_diff_Hf.png
保存： /content/outputs/trend_ACI_with_plants_dist.png
保存： /content/outputs/glass_delta_ACI.png
保存： /content/outputs/raw_diff_ACI.png

所有结果输出目录： /content/outputs
